# Demand prediction for theme parks (specific case of USS)

In [2]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import simpy
from datetime import datetime, timedelta
import os
os.chdir("C:/Users/THAI LOW Jin Yang/data-science-guest-experience/Scripts/Subgroup_B")

## Loading datasets
### Load survey data

What the function does:
- renames question columns for easier reference
- converting wait times to numerical values, taking the median/mean of the range
- adding synthetic event column: USS only has 1 notable special event which is HHN, which is assigned to visitors that came during October - December season. Otherwise, the column is assigned as none. For visitors that forgot the timeline in which they visited, a random value will be assigned.
- handling long wait times: splits attractions that have long wait times into separate observations and assign a baseline wait time which is 90 minutes, for those that felt queueing time was not worth the experience, and 75 for those that do. 
- normalise guest satisfaction score
- process timestamp and extracting date features
- filters for valid attractions, which only keeps popular attractions like: revenge of the mummy, cylon, transformers, etc"
- ensuring required columns are present

In [3]:
def load_survey_data(file_path="../../data/survey.csv"):
    """
    Loads and preprocesses survey data from survey.csv, extracting seasonal information
    and normalizing satisfaction scores. Timestamp is removed in favor of seasonal tagging.

    Args:
        file_path (str): Path to the survey CSV file.

    Returns:
        pd.DataFrame: Preprocessed survey data with 'Season' column added.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} not found. Please provide the survey dataset.")

    df = pd.read_csv(file_path)

    df = df.rename(columns={
        "On a scale of 1-5, how would you rate your overall experience at USS?": "Guest_Satisfaction_Score",
        "How long did you wait in line for rides on average during your visit?": "Wait_Time",
        "Which ride or attraction was your favourite?": "Attraction"
    })

    # Map wait times to numerical estimates
    wait_time_mapping = {
        "Less than 15 mins": 10,
        "15-30 mins": 22.5,
        "30-45 mins": 37.5,
        "45 mins - 1 hr": 52.5,
        "More than 1 hr": 75
    }
    df["Wait_Time"] = df["Wait_Time"].map(wait_time_mapping).fillna(37.5)

    # Generate synthetic Event column
    np.random.seed(42)
    df["Event"] = np.random.choice(['None', 'Special Event'], size=len(df), p=[0.8, 0.2])

    # Handle optional long wait experience data (as before)
    long_wait_df = pd.DataFrame()
    if 'Did you experience any rides with longer-than-expected wait times? If yes, which ride(s)?' in df.columns:
        long_wait_rides = df['Did you experience any rides with longer-than-expected wait times? If yes, which ride(s)?'].str.split(', ', expand=True).stack().reset_index()
        long_wait_rides.columns = ['original_index', 'split_index', 'Attraction']
        long_wait_rides = long_wait_rides[long_wait_rides['Attraction'].notna()]

        queue_worth_col = 'Did you feel that overall, the queuing time was worth the experience of the attraction? '
        unpleasant_col = 'What made your experience with this ride or attraction unpleasant? '

        wait_time_adjusted = []
        for idx in long_wait_rides['original_index']:
            base_wait = 75
            if queue_worth_col in df.columns and df[queue_worth_col].iloc[idx] == 'No':
                base_wait = 90
            if unpleasant_col in df.columns and pd.notna(df[unpleasant_col].iloc[idx]):
                if 'long wait' in str(df[unpleasant_col].iloc[idx]).lower():
                    base_wait += 15
            wait_time_adjusted.append(base_wait)

        long_wait_df = pd.DataFrame({
            'Attraction': long_wait_rides['Attraction'],
            'Event': df['Event'].iloc[long_wait_rides['original_index']].values,
            'Wait_Time': wait_time_adjusted,
            'Guest_Satisfaction_Score': df['Guest_Satisfaction_Score'].iloc[long_wait_rides['original_index']].values,
            'Which part of the year did you visit USS?': df['Which part of the year did you visit USS?'].iloc[long_wait_rides['original_index']].values,
            'Did you purchase the Express Pass?': df['Did you purchase the Express Pass?'].iloc[long_wait_rides['original_index']].values if 'Did you purchase the Express Pass?' in df.columns else [None] * len(long_wait_rides),
            'What was the main purpose of your visit?': df['What was the main purpose of your visit?'].iloc[long_wait_rides['original_index']].values if 'What was the main purpose of your visit?' in df.columns else [None] * len(long_wait_rides),
            'Who did you visit USS with?': df['Who did you visit USS with?'].iloc[long_wait_rides['original_index']].values if 'Who did you visit USS with?' in df.columns else [None] * len(long_wait_rides),
            'Which age group do you belong to?': df['Which age group do you belong to?'].iloc[long_wait_rides['original_index']].values if 'Which age group do you belong to?' in df.columns else [None] * len(long_wait_rides)
        })

    base_df = df[['Attraction', 'Wait_Time', 'Event', 'Guest_Satisfaction_Score',
                  'Which part of the year did you visit USS?', 'Did you purchase the Express Pass?',
                  'What was the main purpose of your visit?', 'Who did you visit USS with?',
                  'Which age group do you belong to?']].copy()

    df_combined = pd.concat([base_df, long_wait_df], ignore_index=True)

    # Normalize Guest Satisfaction
    df_combined["Guest_Satisfaction_Score"] = pd.to_numeric(df_combined["Guest_Satisfaction_Score"], errors="coerce")
    df_combined["Guest_Satisfaction_Score"] = (
        (df_combined["Guest_Satisfaction_Score"] - df_combined["Guest_Satisfaction_Score"].min()) /
        (df_combined["Guest_Satisfaction_Score"].max() - df_combined["Guest_Satisfaction_Score"].min())
    )

    # Assign season based on user input or synthetic if 'Can't recall'
    def assign_season(row):
        season = row['Which part of the year did you visit USS?']
        if season != "Can't recall / Not sure":
            return season
        return np.random.choice(
            ["July - September", "October - December", "January - March", "April - June"],
            p=[0.6, 0.3, 0.05, 0.05]
        )

    df_combined['Season'] = df_combined.apply(assign_season, axis=1)

    # Filter valid attractions
    valid_attractions = [
        "Revenge of the Mummy",
        "Battlestar Galactica: CYLON",
        "Transformers: The Ride",
        "Puss In Boots' Giant Journey",
        "Sesame Street Spaghetti Space Chase"
    ]
    df_combined = df_combined[df_combined['Attraction'].isin(valid_attractions)]

    return df_combined


df_survey = load_survey_data()
print(df_survey.head())

                             Attraction  Wait_Time          Event  \
4   Sesame Street Spaghetti Space Chase       37.5           None   
5                  Revenge of the Mummy       37.5           None   
6                  Revenge of the Mummy       37.5           None   
8   Sesame Street Spaghetti Space Chase       37.5           None   
11                 Revenge of the Mummy       37.5  Special Event   

    Guest_Satisfaction_Score Which part of the year did you visit USS?  \
4                       0.50                   Can't recall / Not sure   
5                       0.75                   Can't recall / Not sure   
6                       0.50                   Can't recall / Not sure   
8                       0.75                        October - December   
11                      0.25                   Can't recall / Not sure   

   Did you purchase the Express Pass?  \
4                                  No   
5                                  No   
6                

### Load IOT data (optional, to answer question 5)

In [12]:
def load_iot_data(file_path="../../data/synthetic_iot_data.csv"):
    """
    Loads synthetic IoT data for demand prediction, adding day_of_week, is_weekend, and is_popular_attraction features.

    Args:
        file_path (str): Path to the synthetic IoT CSV file.

    Returns:
        pd.DataFrame: Preprocessed IoT data.
    """
    if not os.path.exists(file_path):
        print(f"Warning: IoT data file {file_path} not found. Skipping IoT data integration.")
        return None

    df_iot = pd.read_csv(file_path)

    # Convert Timestamp to datetime
    df_iot['Timestamp'] = pd.to_datetime(df_iot['Timestamp'])

    # Add day_of_week feature
    df_iot['day_of_week'] = df_iot['Timestamp'].dt.day_name()

    # Add is_weekend feature (True for Saturday and Sunday)
    df_iot['is_weekend'] = df_iot['day_of_week'].isin(["Saturday", "Sunday"])

    # Define popular attractions
    POPULAR_ATTRACTIONS = {"Revenge of the Mummy", "Battlestar Galactica: CYLON", "Transformers: The Ride"}

    # Add is_popular_attraction feature
    df_iot['is_popular_attraction'] = df_iot['Attraction'].isin(POPULAR_ATTRACTIONS)

    return df_iot

# Example usage
df_iot = load_iot_data()
print(df_iot.head())

            Timestamp                    Attraction    Age_Group  Gender  \
0 2024-12-02 17:13:25  Puss In Boots' Giant Journey       Senior  Female   
1 2024-12-01 13:26:11  Puss In Boots' Giant Journey        Adult    Male   
2 2024-11-29 07:52:28  Puss In Boots' Giant Journey        Child    Male   
3 2024-08-01 16:39:11  Puss In Boots' Giant Journey  Young Adult    Male   
4 2024-05-31 18:22:35          Revenge of the Mummy        Child  Female   

  Loyalty_Member  Check_In_Time  Check_Out_Time  Step_Count  \
0             No              9              14       12775   
1             No             13              16       14102   
2             No             12              17       13212   
3             No              9              16       13017   
4             No             12              17        9916   

   Transaction_Amount  Guest_Satisfaction_Score  Average_Queue_Time  \
0                 171                  3.945533                  51   
1                 217 

### Checking which features are important to predict demand for IOT data

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt

df_iot = load_iot_data()
correlation_matrix = df_iot.corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.show()


ValueError: could not convert string to float: "Puss In Boots' Giant Journey"

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

features = ['Guest_Satisfaction_Score', 'Average_Queue_Time', 'Check_In_Time', 
            'Check_Out_Time', 'Temperature', 'Rainfall', 'Humidity']
X = df_iot[features]
y = df_iot['Number_of_People_in_Queue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

feature_importance = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
print(feature_importance)


Rainfall                    0.191108
Humidity                    0.181672
Temperature                 0.175049
Guest_Satisfaction_Score    0.172186
Average_Queue_Time          0.147287
Check_Out_Time              0.070059
Check_In_Time               0.062638
dtype: float64


## Load weather data

In [7]:
def fetch_weather_data(file_path="../../data/singapore_seasonal_weather.csv"):
    """
    Fetches or loads seasonal weather data for all months of 2024,
    calculates seasonal averages, and saves the result for reuse.

    Returns:
        pd.DataFrame: Weather data averaged by season.
    """
    import os
    import requests
    import pandas as pd

    if os.path.exists(file_path):
        print(f"✅ Loaded existing weather data from: {file_path}")
        return pd.read_csv(file_path)

    print("📡 Fetching weather data from API...")

    base_url = "https://api.data.gov.sg/v1/environment/"
    weather_types = ["rainfall", "air-temperature", "relative-humidity", "wind-speed"]
    months = [f"2024-{str(m).zfill(2)}-15" for m in range(1, 13)]
    month_names = [datetime.strptime(m, "%Y-%m-%d").strftime("%B") for m in months]

    all_data = []

    for date_str, month_name in zip(months, month_names):
        print(f"Fetching data for: {date_str}")
        daily_data = {"month": month_name}

        for weather_type in weather_types:
            url = f"{base_url}{weather_type}"
            params = {"date": date_str}
            response = requests.get(url, params=params)

            if response.status_code == 200:
                try:
                    data = response.json()
                    readings = data["items"][0]["readings"]
                    avg_value = sum(d["value"] for d in readings) / len(readings)
                    daily_data[weather_type] = avg_value
                except (KeyError, IndexError):
                    print(f"⚠️ Missing data for {weather_type} on {date_str}")
                    daily_data[weather_type] = None
            else:
                print(f"❌ Error fetching {weather_type} for {date_str}: {response.status_code}")
                daily_data[weather_type] = None

        all_data.append(daily_data)

    df = pd.DataFrame(all_data)

    # Map months to seasons
    month_to_season = {
        "January": "January - March", "February": "January - March", "March": "January - March",
        "April": "April - June", "May": "April - June", "June": "April - June",
        "July": "July - September", "August": "July - September", "September": "July - September",
        "October": "October - December", "November": "October - December", "December": "October - December"
    }
    df["Season"] = df["month"].map(month_to_season)

    # Average by season
    df_seasonal = df.groupby("Season").agg({
        "rainfall": "mean",
        "air-temperature": "mean",
        "relative-humidity": "mean",
        "wind-speed": "mean"
    }).reset_index()

    df_seasonal.rename(columns={
        "air-temperature": "air_temperature",
        "relative-humidity": "relative_humidity",
        "wind-speed": "wind_speed"
    }, inplace=True)

    # Save to disk
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    df_seasonal.to_csv(file_path, index=False)
    print(f"✅ Saved seasonal weather data to: {file_path}")

    return df_seasonal

In [8]:
df_weather = fetch_weather_data("../../data/singapore_seasonal_weather.csv")
print(df_weather.head())

📡 Fetching weather data from API...
Fetching data for: 2024-01-15
Fetching data for: 2024-02-15
Fetching data for: 2024-03-15
Fetching data for: 2024-04-15
Fetching data for: 2024-05-15
Fetching data for: 2024-06-15
Fetching data for: 2024-07-15
Fetching data for: 2024-08-15
Fetching data for: 2024-09-15
Fetching data for: 2024-10-15
Fetching data for: 2024-11-15
Fetching data for: 2024-12-15
✅ Saved seasonal weather data to: ../../data/singapore_seasonal_weather.csv
               Season  rainfall  air_temperature  relative_humidity  \
0        April - June  0.000000        27.910354          86.727778   
1     January - March  0.001093        27.134091          81.434444   
2    July - September  0.000000        27.970971          78.282112   
3  October - December  0.000000        27.033810          83.870873   

   wind_speed  
0    2.330833  
1    4.568889  
2    2.646852  
3    2.230736  


## Merging datasets
### Merging survey and weather data (to analyse the absence of IOT data to feed into the model)

In [24]:
def merge_survey_weather_iot(survey_df, weather_df, iot_df=None):
    """
    Merges survey data with seasonal weather data, and appends IoT data if provided.
    
    Args:
        survey_df (pd.DataFrame): Survey data with a 'Season' column.
        weather_df (pd.DataFrame): Seasonal weather data with 'Season' column.
        iot_df (pd.DataFrame, optional): IoT data (should contain 'Season' column).

    Returns:
        pd.DataFrame: Combined dataset (survey + weather [+ iot if provided]).
    """
    import pandas as pd
    import numpy as np

    # Merge survey with weather data
    merged_survey = pd.merge(survey_df, weather_df, on='Season', how='left')

    if iot_df is None:
        return merged_survey

    # Ensure 'Season' exists in IoT data
    if 'Season' not in iot_df.columns:
        print("⚠️ 'Season' column missing in IoT data. Assigning season synthetically...")
        if 'Timestamp' in iot_df.columns:
            iot_df['Timestamp'] = pd.to_datetime(iot_df['Timestamp'])
            month_to_season = {
                1: "January - March", 2: "January - March", 3: "January - March",
                4: "April - June", 5: "April - June", 6: "April - June",
                7: "July - September", 8: "July - September", 9: "July - September",
                10: "October - December", 11: "October - December", 12: "October - December"
            }
            iot_df['Season'] = iot_df['Timestamp'].dt.month.map(month_to_season)
        else:
            iot_df['Season'] = np.random.choice(
                ["January - March", "April - June", "July - September", "October - December"],
                size=len(iot_df),
                p=[0.1, 0.1, 0.4, 0.4]
            )

    # Merge IoT with weather
    merged_iot = pd.merge(iot_df, weather_df, on='Season', how='left')

    # Append both datasets (not inner join, preserve all columns)
    combined = pd.concat([merged_survey, merged_iot], ignore_index=True, join='outer')

    return combined


### Merged dataset without IOT data

In [25]:
df_combined = merge_survey_weather_iot(df_survey, df_weather)
print(df_combined.head())
print(df_combined.columns.tolist())

                            Attraction  Wait_Time          Event  \
0  Sesame Street Spaghetti Space Chase       37.5           None   
1                 Revenge of the Mummy       37.5           None   
2                 Revenge of the Mummy       37.5           None   
3  Sesame Street Spaghetti Space Chase       37.5           None   
4                 Revenge of the Mummy       37.5  Special Event   

   Guest_Satisfaction_Score Which part of the year did you visit USS?  \
0                      0.50                   Can't recall / Not sure   
1                      0.75                   Can't recall / Not sure   
2                      0.50                   Can't recall / Not sure   
3                      0.75                        October - December   
4                      0.25                   Can't recall / Not sure   

  Did you purchase the Express Pass? What was the main purpose of your visit?  \
0                                 No                            Family 

###  Merged dataset with IOT data

In [26]:
df_all_combined = merge_survey_weather_iot(df_survey, df_weather, df_iot)
print(df_all_combined.head())
print(df_all_combined.columns.tolist())

                            Attraction  Wait_Time          Event  \
0  Sesame Street Spaghetti Space Chase       37.5           None   
1                 Revenge of the Mummy       37.5           None   
2                 Revenge of the Mummy       37.5           None   
3  Sesame Street Spaghetti Space Chase       37.5           None   
4                 Revenge of the Mummy       37.5  Special Event   

   Guest_Satisfaction_Score Which part of the year did you visit USS?  \
0                      0.50                   Can't recall / Not sure   
1                      0.75                   Can't recall / Not sure   
2                      0.50                   Can't recall / Not sure   
3                      0.75                        October - December   
4                      0.25                   Can't recall / Not sure   

  Did you purchase the Express Pass? What was the main purpose of your visit?  \
0                                 No                            Family 

# Modelling with XGBoost

In [29]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def train_demand_model_flexible(df, target='Number_of_People_in_Queue'):
    """
    Trains an XGBoost model that adapts to the columns available in the dataset,
    including fallback handling for missing values due to merged sources.

    Args:
        df (pd.DataFrame): Merged dataset (survey + weather [+ iot]).
        target (str): Name of the target column.

    Returns:
        model (XGBRegressor): Trained model.
        metrics (dict): Evaluation results.
    """

    # Full list of possible features (IoT + survey + weather)
    potential_features = [
        'Season', 'Attraction', 'Wait_Time', 'Guest_Satisfaction_Score',
        'rainfall', 'relative_humidity', 'air_temperature',
        'Average_Queue_Time', 'Check_In_Time', 'Check_Out_Time'
    ]

    # --- Filter features that actually exist and are not fully null ---
    available_features = [col for col in potential_features if col in df.columns and df[col].notna().any()]
    print(f"🧮 Using features: {available_features}")

    # --- Drop rows where the target is missing ---
    df = df[df[target].notna()]
    if df.empty:
        raise ValueError(f"❌ No rows with valid target '{target}'.")

    # --- Fill missing values in features ---
    df = df[available_features + [target]].copy()
    for col in available_features:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna("Unknown")
        else:
            df[col] = df[col].fillna(df[col].mean())  # Use mean for numeric cols

    # --- Encode categoricals ---
    for col in ['Season', 'Attraction']:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))

    # --- Split data ---
    X = df[available_features]
    y = df[target]

    if len(df) < 5:
        raise ValueError("❌ Not enough samples to split. Need at least 5 rows.")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # --- Train XGBoost ---
    model = XGBRegressor(
        random_state=42,
        n_estimators=50,
        max_depth=4,
        learning_rate=0.1,
        verbosity=0
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # --- Evaluation ---
    metrics = {
        'R² Score': r2_score(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred)
    }

    print("✅ Model trained successfully.")
    print("📊 Evaluation:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

    return model, metrics


In [30]:
model, metrics = train_demand_model_flexible(df_all_combined)

🧮 Using features: ['Season', 'Attraction', 'Wait_Time', 'Guest_Satisfaction_Score', 'rainfall', 'relative_humidity', 'air_temperature', 'Average_Queue_Time', 'Check_In_Time', 'Check_Out_Time']
✅ Model trained successfully.
📊 Evaluation:
R² Score: 0.0297
RMSE: 85.4621
MAE: 74.6309
